In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import itertools
import xgboost 
import lightgbm
import networkx
import nodevectors
import os,gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/hokage/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/hokage/.local/lib/python3.6/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [ ]:
train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
print(train.shape, test.shape)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
print(train.isna().any().sum(), test.isna().any().sum())

Cool, we have no missing values and all columns are numeric type.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,8))
sns.countplot(train.open_channels, ax=ax[0])
sns.distplot(train.open_channels, ax=ax[1])

Classification. Higher number of open channels are less frequent. Let's check how open_channels vary with time. 

In [ ]:
# plt.figure(figsize=(20,8))
# sns.scatterplot(x='time', y='open_channels', data=train)

Insights:
1. Values 0 and 1 for open_channels is very frequent as compared to other values.
2. Values >= 6 occur between 200s and 250s and then between 450s and 500s. Seasonality? Maybe
3. Values 2 and 3 occur between 150s and 300s and then between 350s and 500s. 

In [ ]:
# plt.figure(figsize=(20,8))
# sns.lineplot(x='time', y='signal', hue='open_channels', data=train[train['time'] < 100])

Cool. So, we can say that a single channel is opened only when the value of signal > -2.

In [ ]:
# plt.figure(figsize=(20,8))
# sns.lineplot(x='time', y='signal', hue='open_channels', data=train[(train['time'] > 100) & (train['time'] < 200)])

Clearly, the number of open channels increases with increase in signal. This graph is in tune with our time vs open_channels graph.

In [ ]:
# plt.figure(figsize=(20,8))
# sns.lineplot(x='time', y='signal', hue='open_channels', data=train[(train['time'] > 200) & (train['time'] < 300)])

In [ ]:
# plt.figure(figsize=(20,8))
# sns.lineplot(x='time', y='signal', hue='open_channels', data=train[(train['time'] > 300) & (train['time'] < 400)])

In [ ]:
# plt.figure(figsize=(20,8))
# sns.lineplot(x='time', y='signal', hue='open_channels', data=train[train['time'] > 400])

In [ ]:
train.describe()

## Feature Engineering

In [ ]:
G = networkx.Graph()
G

In [ ]:
temp = train[['signal', 'open_channels']]
temp

In [ ]:
temp['signal'] = temp['signal'].round(2)
temp['signal'].nunique()

In [ ]:
temp['signal'] = temp['signal'].astype(str)
temp

In [ ]:
G.add_nodes_from(temp['signal'].values, signal=True)
print(len(G.nodes()))

In [ ]:
temp['open_channels'] = temp['open_channels'].apply(lambda x : -99 if x ==0 else x * -999)
temp

In [ ]:
G.add_nodes_from(temp['open_channels'].values, channel=True)
print(len(G.nodes()))

In [ ]:
G.add_edges_from(temp.values)
print(len(G.nodes()), len(G.edges()))

In [ ]:
def createEdgesBetweenSignalsWithSameOpenChannel(G):
    channel_nodes = networkx.get_node_attributes(G, 'channel')
    channel_nodes = list(channel_nodes.keys())
    edges_to_be_created = []

    for i in channel_nodes:
        if(len([x for x in G.neighbors(i)]) > 1):
            edges_to_be_created.append([x for x in G.neighbors(i)])
            
    for i in edges_to_be_created:
        for j in itertools.combinations(i,2):
            G.add_edge(*j)
            
    return G 

In [ ]:
G = createEdgesBetweenSignalsWithSameOpenChannel(G)
print(len(G.edges()))

In [ ]:
channel_nodes = networkx.get_node_attributes(G, 'channel')
channel_nodes = list(channel_nodes.keys())
G.remove_nodes_from(channel_nodes)

In [ ]:
embed_size = 128

n2v = nodevectors.Node2Vec(
    walklen=32,
    epochs=10,
    return_weight=1,
    neighbor_weight=1.0,
    n_components=128,
    w2vparams={'window': 10,
               'min_count':1
              }
)
n2v.fit(G, verbose=True)

In [ ]:
%%time
nodes = [i for i in n2v.model.wv.vocab]
embeddings = np.array([n2v.model.wv[x] for x in nodes])

In [ ]:
embedding_df = pd.DataFrame()
embedding_df['signal'] = nodes
embedding_df['embed'] = list(embeddings)
embedding_df

In [ ]:
del G, temp
gc.collect()

In [ ]:
embed_cols = [f"embed{i}" for i in range(128)]
embed_cols

df3 = pd.DataFrame(embedding_df['embed'].values.tolist(), columns=embed_cols)
df3

In [ ]:
embedding_df = embedding_df.join(df3)
embedding_df = embedding_df.drop(['embed'], axis=1)
embedding_df

In [ ]:
train.info()

In [ ]:
embedding_df['signal'] = embedding_df['signal'].astype('float64')
embedding_df

In [ ]:
train['signalRound2'] = train['signal'].round(2)
train

In [ ]:
train = train.merge(embedding_df, left_on='signalRound2', right_on='signal', how='left')
train

In [ ]:
del embedding_df
gc.collect()

## Modeling

In [ ]:
KFOLDS = 5
cv = KFold(n_splits=KFOLDS, shuffle=True, random_state=108)

In [ ]:
# xgb = xgboost.XGBClassifier(tree_method='hist', objective='multi:softmax')
param_grid ={
    'learning_rate': [0.05],
    'n_estimators':[100],
    }

In [ ]:
lgb = lightgbm.LGBMClassifier(objective='multiclass')

In [ ]:
clf = GridSearchCV(
        estimator=lgb,
        param_grid=param_grid,
        cv=cv,
        iid=True,
        return_train_score=True,
        scoring='f1_macro',
        verbose=100
    )

In [ ]:
clf.fit(train.drop(['time', 'signal_y', 'open_channels', ''],axis=1),y)

In [ ]:
plt.figure(figsize=(20,6))
sns.barplot(x=clf.best_estimator_.feature_importances_, y=X.columns)

In [ ]:
X_test = all_data[all_data['train'] == 0]
X_test

In [ ]:
predictions = clf.predict(X_test.drop(['train', 'time'], axis=1))
predictions

In [ ]:
plt.figure(figsize=(8,6))
sns.countplot(predictions)

In [ ]:
test['open_channels'] = predictions

In [ ]:
test = test[['time', 'open_channels']]
test.to_csv('submission.csv', index=False, float_format='%.4f')